# Investigation

This will be my attempt at creating something similar to the original notebook.
A few things to think about:

- After doing the initial recreation, let's look at the Kaggle tutorials and add in other data points
  - Close - n datapoints? So we're not just using the very latest close price
  - High/Low/Open - see if these are useful
  - Tick volume - see if this is useful
- As well as looking at root mean squared error, lets look at statistics
  - When model predict a move of more than 1, 2, 3 ATR in next 10, 15 candles, how often does this actually happen?
    - When it's correct, how often is there a move in the opposite direction first that was not seen by model
  - When it predicts a positive or negative move, how often is it actually correct?
    - Look at this over different candle counts. ie. Candle 1, 10, 15

Hopefully, we will find models that are better than the original but there's not going to be a model we can use for native price forcasting. But perhaps we can use it to inform other prediction algorithms. ie. If we have an algorithm that identifies support and resistance areas, and the model is good a predicting the direction of the movement then it can be used to inform on where to place a trade even if it can't predict the price.

### Note

In this repo I am using a custom module `malfoy.metatrader` that I built on top of the `MetaTrader5` package. This interfaces directly with MetaTrader to pull up-to-date data. I hope to eventually make this open-source but for now it's a private module. I have also saved the data used in `./data/new` so it can be reused.

In [10]:
# Import modules

import pandas as pd

## Variables

These core variables are used throughout the scripts. The instrument and granularity variables can be used to change the target of predictions. These correspond to the data found in `./data/new/`

In [11]:
# Set script variables
inputDataFolder = "./input/new"
instrument = "BTCUSD"
granularity = "M5"

In [ ]:
# Import data

rawDataFilePath = "{0}/{1}_{2}.csv".format(inputDataFolder, instrument, granularity)
rawData = pd.read_csv(rawDataFilePath)

rawData.head()